In [1]:
import config
import json
import requests
import vulture_scraper
api_key = config.API_key

In [2]:
## returns data from Movie DB's list of popular movies. updated daily 
def get_popular(api_key):

    url = "https://api.themoviedb.org/3/movie/popular?api_key={}&language=en-US".format(api_key)
    return requests.get(url).json()

In [3]:
## returns only the titles from Movie DB's list of popular movies 
def get_popular_titles(api_key):
    data = get_popular(api_key)
    return [x['title'] for x in data['results']]

get_popular_titles(api_key)[:5]

['Joker',
 'Maleficent: Mistress of Evil',
 'Terminator: Dark Fate',
 'Fast & Furious Presents: Hobbs & Shaw',
 'The Lion King']

In [4]:
## returns data from Movie DB's list of top rated movies 
def get_top_rated(api_key):
    url = "https://api.themoviedb.org/3/movie/top_rated?api_key={}&language=en-US".format(api_key)
    return requests.get(url).json()

In [5]:
## returns list of IDs from Movie DB's top rated movies 
def get_top_rated_ids(api_key):
    data = get_top_rated(api_key)
    return [x['id'] for x in data['results']]


In [6]:
## returns full details when given a list of movie IDs
def get_movies(ids):
    data = []
    for i in ids:
        url = "https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US".format(i,api_key)
        data.append(requests.get(url).json())
    return data

In [7]:
## returns movie data when given an API key and list of movie titles 

def search_movies(api_key, list_of_movies):
    data = []
    for movie in list_of_movies:
        url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
        data.append(requests.get(url).json())
    return data


In [9]:
## search Movie DB for the best movies on Hulu according to Vulture 

hulu_movies = search_movies(api_key, vulture_scraper.best_hulu)

In [10]:
hulu_movies[0]

{'page': 1,
 'total_results': 1,
 'total_pages': 1,
 'results': [{'popularity': 31.586,
   'vote_count': 4273,
   'video': False,
   'poster_path': '/AkJQpZp9WoNdj7pLYSj1L0RcMMN.jpg',
   'id': 353081,
   'adult': False,
   'backdrop_path': '/aw4FOsWr2FY373nKSxbpNi3fz4F.jpg',
   'original_language': 'en',
   'original_title': 'Mission: Impossible - Fallout',
   'genre_ids': [28, 12],
   'title': 'Mission: Impossible - Fallout',
   'vote_average': 7.3,
   'overview': 'When an IMF mission ends badly, the world is faced with dire consequences. As Ethan Hunt takes it upon himself to fulfill his original briefing, the CIA begin to question his loyalty and his motives. The IMF team find themselves in a race against time, hunted by assassins while trying to prevent a global catastrophe.',
   'release_date': '2018-07-27'}]}